In [33]:
import sys, heapq

In [29]:
class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action

class PrioNode():
    def __init__(self, state, parent, action, priority):
        self.state = state
        self.parent = parent
        self.action = action
        self.priority = priority
    
    def __lt__(self, other):
        return self.priority < other.priority

In [38]:
class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)
    
    def empty(self):
        return len(self.frontier) == 0

    def size(self):
        return len(self.frontier)
        
    def pop(self):
        if self.empty():
            raise Exception("fontier is empty")
        else:
            return self.frontier.pop(-1)

        
class QueueFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)
    
    def empty(self):
        return len(self.frontier) == 0

    def size(self):
        return len(self.frontier)
        
    def pop(self):
        if self.empty():
            raise Exception("fontier is empty")
        else:
            return self.frontier.pop(0)

        
class HeapFrontier():
    def __init__(self):
        self.frontier = []
    def add(self, node: PrioNode):
        heapq.heappush(self.frontier, node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)
    
    def empty(self):
        return len(self.frontier) == 0

    def size(self):
        return len(self.frontier)
        
    def pop(self):
        if self.empty():
            raise Exception("fontier is empty")
        else:
            return heapq.heappop(self.frontier)


In [47]:
class Maze():
    def __init__(self, filename):
        # Read file
        with open(filename, 'r') as f:
            contents = f.read()
        
        if contents.count('A') != 1:
            raise Exception("Maze needs a single startpoint")
        if contents.count('B') != 1:
            raise Exception("Maze needs a single endpoint")

        contents = contents.splitlines()
        # Record height and width of maze
        self.height = len(contents)
        self.width = max(len(line) for line in contents)

        # Add context
        self.walls = [] # 2D bool array
        for r in range(self.height):
            row = []
            for c in range(self.width):
                # Try block as not all mazes will be rectangular
                try:
                    if contents[r][c] == '#':
                        row.append(True)
                    elif contents[r][c] == ' ':
                        row.append(False)
                    elif contents[r][c] == 'A':
                        self.start = (r,c)
                        row.append(False)
                    elif contents[r][c] == 'B':
                        self.goal = (r,c)
                        row.append(False)
                    else:
                        row.append(True)
                except IndexError:
                    row.append(True)
            self.walls.append(row)
        self.solution = None

        
    def print(self):
        solution = self.solution[1] if self.solution is not None else None
        for row in range(self.height):
            for col in range(self.width):
                if (row,col) == self.start:
                    print('A', end='')
                elif (row,col) == self.goal:
                    print('B', end='')
                elif self.walls[row][col] == True:
                    print('#', end='')
                elif solution is not None and (row,col) in solution:
                    print('*', end='')
                else:
                    print(' ', end='')
            print()
        print()

        
    def neighbors(self, state):
        row, col = state
        # All possible actions
        candidates = [
            ('up', (row-1, col)),
            ('down', (row+1, col)),
            ('left', (row, col-1)),
            ('right', (row, col+1)),
        ]
        # Validate actions
        possible = []
        for action, (row, col) in candidates:
            try:
                if not self.walls[row][col]:
                    possible.append((action, (row,col)))
            except IndexError:
                continue
        return possible
    
    
    def solve(self):
        # Keep track of how many states have been explored (performance measure)
        self.num_explored = 0

        # Initialize frontier to the starting position
        start = Node(state=self.start, parent=None, action=None)
        frontier = QueueFrontier()
        frontier.add(start)

        # Initialize explored set 
        self.explored = set()

        # Search for solution
        while True:
            if frontier.empty():
                raise Exception("no solution")
            node = frontier.pop()
            self.num_explored += 1
            if node.state == self.goal:
                actions = []
                cells = []

                # Follow breadcrumbs to remake solution path
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                # B -> A --> A -> B
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                return
            # Add to explored
            self.explored.add(node.state)

            # Add neighbors to frontier
            for action, state in self.neighbors(node.state):
                if not frontier.contains_state(state) and state not in self.explored:
                    child = Node(state=state, parent=node, action=action)
                    frontier.add(child)


In [52]:
maze = Maze('maze1.txt')

In [53]:
maze.print()

########################################################################
########################################################################
########################################################################
#            ##                                                        #
#   A         ##       #                  ##                           #
#              ##      #        ##       ##                       B    #
#        ##            #         ##     ##                             #
#         ##         #            ##   ##              #################
#          ##        #           ##     ##             #################
#                    #          ##       ##            #################
#            #                 ##                      #################
#                                                      #################
#            #            ##                           #################
#            #             ##                      

In [56]:
maze.solve()

In [57]:
maze.solution

(['down',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'down',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'up',
  'up',
  'up',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'down',
  'down',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right'],
 [(5, 4),
  (5, 5),
  (5, 6),
  (5, 7),
  (5, 8),
  (5, 9),
  (5, 10),
  (5, 11),
  (6, 11),
  (6, 12),
  (6, 13),
  (6, 14),
  (6, 15),
  (6, 16),
  (6, 17),
  (5, 17),
  (4, 17),
  (3, 17),
  (3, 18),
  (3, 19),
  (3, 20),
  (3, 21),
  (3, 22),
  (3, 

In [55]:
maze.print()

########################################################################
########################################################################
########################################################################
#            ##  ****************************                          #
#   A         ## *     #                  ##*                          #
#   ********   ##*     #        ##       ## **********************B    #
#        ##*******     #         ##     ##                             #
#         ##         #            ##   ##              #################
#          ##        #           ##     ##             #################
#                    #          ##       ##            #################
#            #                 ##                      #################
#                                                      #################
#            #            ##                           #################
#            #             ##                      